In [20]:
import platform
print(platform.python_version())
print(spacy.__version__)
print(json.__version__)

3.7.8
2.3.2
2.0.9


In [29]:
import json
import logging
import sys

def tsv_to_json(input_path, output_path, unknown_label):
    try:
        input_file = open(input_path, 'r', encoding='UTF-8')
        output_file = open(output_path, 'w', encoding='UTF-8')
        data_dict = {}
        annotations = []
        label_dict = {}
        s = ''
        start = 0
        for line in input_file:
            if line[0:len(line)-1] != '.\tO':
                word, entity = line.split('\t')
                s += word+" "
                entity = entity[:len(entity)-1]
                if entity != unknown_label and len(entity) != 1:
                    d = {}
                    d['text'] = word
                    d['start'] = start
                    d['end'] = start+len(word)-1
                    try:
                        label_dict[entity].append(d)
                    except:
                        label_dict[entity] = []
                        label_dict[entity].append(d)
                start += len(word)+1
            else:
                data_dict['content'] = s
                s = ''
                label_list = []
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text'] != ''):
                            l = [ents, label_dict[ents][i]]
                            for j in range(i+1, len(label_dict[ents])):
                                if(label_dict[ents][i]['text'] == label_dict[ents][j]['text']):
                                    di = {}
                                    di['start'] = label_dict[ents][j]['start']
                                    di['end'] = label_dict[ents][j]['end']
                                    di['text'] = label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text'] = ''
                            label_list.append(l)

                for entities in label_list:
                    label = {}
                    label['label'] = [entities[0]]
                    label['points'] = entities[1:]
                    annotations.append(label)
                data_dict['annotation'] = annotations
                annotations = []
                json.dump(data_dict, output_file, ensure_ascii=False)
                output_file.write('\n')
                data_dict = {}
                label_dict = {}
                start = 0                
    except Exception as e:
        logging.exception("Unable to process file\nerror = " + str(e))
        raise e

tsv_to_json("med-corpus.tsv", 'med-corpus.json', 'abc')

In [30]:
import spacy
import json
import random
# Read json file and transform it to array 
def transform_json(json_file_path):
    corpus = open(json_file_path, 'r') 
    lines = corpus.readlines() 
  
    training_data = [] #array of train data with marked up medical entities
    count = 0
    # read lines from file and parsing them into special array
    for line in lines: 
        res = json.loads(line)   
        text = res['content']
        entities = []
        for annotation in res['annotation']:
            point = annotation['points'][0]
            labels = annotation['label']
            if not isinstance(labels, list):
                labels = [labels]
            for label in labels:
                entities.append((point['start'], point['end'] + 1 ,label))
        training_data.append((text, {"entities" : entities}))
    return training_data
training_data = transform_json('med-corpus.json')

In [31]:
print(training_data)

imantadin so bald wie möglich wieder abgesetzt werden , in der Regel nach 3 bis 5 Behandlungstagen , oder innerhalb von 24 bis 48 Stunden nach Abklingen der Symptomatik ', {'entities': [(18, 26, 'b-med'), (31, 40, 'b-med'), (46, 56, 'b-med'), (204, 215, 'b-med')]}), ('Die neueren Neuraminidasehemmer sind für die Behandlung von Patienten zugelassen , die ein Jahr oder älter sind Oseltamivir oder für Patienten , die sieben Jahre oder älter sind Zanamivir ', {'entities': [(12, 31, 'b-med'), (45, 55, 'b-med'), (60, 69, 'b-med'), (112, 123, 'b-med'), (178, 187, 'b-med')]}), ('Sie sind bei Patienten mit einer komplikationslos verlaufenden akuten Erkrankung indiziert , wenn die Symptomatik noch nicht längerals zwei Tage besteht ', {'entities': [(13, 22, 'b-med'), (70, 80, 'b-med'), (102, 113, 'b-med')]}), ('Die empfohlene Therapiedauer beträgt für beide Medikamente fünf Tage ', {'entities': [(47, 58, 'b-med')]}), ('Theoretisch könnte jedes menschliche Wesen von diesem neuen Virus infiziert we

In [33]:
import warnings

def train_spacy(data,iterations):    
    nlp = spacy.blank('de')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe("ner")   

    # add labels
    for _, annotations in data:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
         # show warnings for misaligned entity spans once
        warnings.filterwarnings("once", category=UserWarning, module='spacy')
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(data)
            losses = {}
            for text, annotations in data:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(training_data, 20)

# Save our trained Model
modelfile = 'models/' + input("Enter your Model Name: ")
prdnlp.to_disk(modelfile)

Statring iteration 0
{'ner': 942.5102391913283}
Statring iteration 1
{'ner': 451.30299259373726}
Statring iteration 2
{'ner': 284.5659009561343}
Statring iteration 3
{'ner': 219.86949792512678}
Statring iteration 4
{'ner': 191.51629554363748}
Statring iteration 5
{'ner': 127.20699251842947}
Statring iteration 6
{'ner': 134.4207719777719}
Statring iteration 7
{'ner': 141.02930733889778}
Statring iteration 8
{'ner': 87.37404514935015}
Statring iteration 9
{'ner': 70.07034648682334}
Statring iteration 10
{'ner': 83.06014916951477}
Statring iteration 11
{'ner': 76.7618093471189}
Statring iteration 12
{'ner': 44.04627077672344}
Statring iteration 13
{'ner': 98.77199549301368}
Statring iteration 14
{'ner': 61.72500883349216}
Statring iteration 15
{'ner': 62.881098124164446}
Statring iteration 16
{'ner': 54.43484834127341}
Statring iteration 17
{'ner': 61.40525640411748}
Statring iteration 18
{'ner': 35.345661456095364}
Statring iteration 19
{'ner': 23.136873534381138}


In [35]:
#Test your text
test_text = input("Enter your testing text: ")
doc = prdnlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Gesundheitssystems 108 126 b-med
Ausnahmeregelungen 183 201 b-med
Apotheken 411 420 b-med
Medikamenten 437 449 b-med
Geldgier 510 518 b-med
Arzneimitteln 549 562 b-med
Arzneimittelversorgung 644 666 b-med
Pandemien 685 694 b-med
